<a href="https://colab.research.google.com/github/Kkuntal990/CTW_2020/blob/master/TRRS_PCA_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
import h5py
f = h5py.File("/content/drive/My Drive/CTW_2019/h_Estimated_CTW_Train.h5")
H = f['h_Estimated'][:].T
f.close()
f = h5py.File("/content/drive/My Drive/CTW_2019/r_Position_CTW_Train.h5")
pos = f["r_Position"][:].T
f.close()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  


OSError: ignored

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(H, pos, test_size=0.1, random_state=42)
del H

In [ ]:
X_tr, X_fd, y_tr, y_fd = train_test_split(X_train, y_train, test_size=0.1, random_state=54)

In [ ]:
import numpy as np
X_train = np.sqrt(X_train[:,:,:,0]**2 + X_train[:,:,:,1]**2)

In [ ]:
from pickle import load
pca = load(open("/content/drive/My Drive/CTW_2019/PCA_Unsupervised.sav" , "rb"))

In [ ]:
from tensorflow.keras.models import Model,Sequential , load_model
best = load_model("/content/drive/My Drive/CTW_2019/Best_PCA_CNN.h5")

In [ ]:
X_train = X_train.reshape(-1,924)
from sklearn.decomposition import PCA
X_train = pca.transform(X_train)
X_train = X_train.reshape(-1 , 16,66)
X_train = X_train.reshape(X_train.shape[0],-1,1)
pos_cnn = best.predict(X_train)
del X_train
pos_cnn , _ = train_test_split(pos_cnn, test_size=0.1, random_state=54)

In [ ]:
pos_cnn.shape

(14163, 3)

In [ ]:
class TRRS:
  def __init__(self, CFR , CFR_Pos):
    self.CFR = CFR
    self.CFR_Pos = CFR_Pos
    import numpy as np
  
  def get_fingerprint_subset(self , pos , radius = 1):
    out = []
    for i in range(len(self.CFR_Pos)):
      if np.sqrt(np.sum((pos-self.CFR_Pos[i])**2)) < radius:
        out.append(i)
    return np.array(out)

  def predict(self , H2 , Pos2=[] , radius=1):
      if len(Pos2)==0:
        idx = range(len(self.CFR))
      else:
        idx = self.get_fingerprint_subset(Pos2 , radius)
      H1 = self.CFR[idx]
      pos = self.CFR_Pos[idx]
      Yd1 = np.sum((H1*H1)[:,:,:,0] + (H1*H1)[:,:,:,1] , axis=2)
      Yd2 = np.sum((H2*H2)[:,:,0] + (H2*H2)[:,:,1] , axis=1)
      #print((Yd1*Yd2).shape)

      H11 = H1[:,:,:,0] + H1[:,:,:,1] * 1j
      H22 = H2[:,:,0] - H2[:,:,1] * 1j
      G = H11 * H22
      #print(G.shape)
      Nser = 1024
      Nu = 924
      G = np.concatenate((G, np.zeros((H1.shape[0],16,Nser-Nu))), axis=2)
      #print(G.shape)

      g = np.fft.fftn(G, axes=(2,))
      #print(g.shape)

      phi_d = np.ndarray.max((np.square(np.abs(g))), axis=2)
      phi_d = np.divide(phi_d,(Yd1*Yd2))
      #print(phi_d.shape)

      w_d = np.sqrt((Yd1 * Yd2))
      w_d = np.transpose(w_d)/(np.sqrt(np.sum(Yd1, axis=1)) * np.sum(Yd2))
      w_d = np.transpose(w_d)
      #print(w_d.shape)
      
      TRRS = np.sum(np.multiply(w_d,np.sqrt(phi_d)), axis=1)
      TRRS = np.square(TRRS)

      idx = np.argmax(TRRS)

      return pos[idx,:]

  

In [ ]:
trrs = TRRS(X_fd , y_fd)

In [ ]:
import numpy as np
pos_trrs = np.zeros(pos_cnn.shape)

In [ ]:
import time
starttime = time.time()
for i in range(len(X_tr)):

  pos_trrs[i] = trrs.predict(X_tr[i] , pos_cnn[i] , radius = 0.15)
  if((i+1)%50==0):
    endtime = time.time()
    print("At i = " , i+1 , "Time :" , endtime-starttime)
    starttime = time.time()

At i =  50 Time : 1.970038652420044


IndexError: ignored